## Requirements

In [ ]:
!pip install feedparser pandas sacremoses  > /dev/null 2>&1
!pip install deepl

import requests
import feedparser
import pandas as pd
from datetime import datetime, timedelta
import time
import urllib.parse
import re
from bs4 import BeautifulSoup
from transformers import MarianMTModel, MarianTokenizer
import deepl
import time

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


#Step 1: Online News Data Collection

##Step 1.a: Fetch Data with using newsapi



In [ ]:
def preprocess_text(text):
    # Use regex to filter out non-alphanumeric characters
    if text:
        return re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return ""

def fetch_turkey_news(api_key, from_date='2024-11-29', language='en', max_pages=5, file_path='/content/drive/MyDrive/Projects/Deeplearning/all_news.pkl'):
    """
    Fetches news articles related to Turkey using NewsAPI and preprocesses the descriptions.
    """
    turkey_keywords = [
        "Turkey", "Türkiye", "Ankara", "Istanbul", "İzmir", "Turkiye"
    ]

    all_descriptions = []

    for keyword in turkey_keywords:
        print(f"Fetching news for keyword: {keyword}")

        encoded_keyword = urllib.parse.quote(keyword)

        for page in range(1, max_pages + 1):
            # Construct the API URL for the current page and keyword
            url = f'https://newsapi.org/v2/everything?q={encoded_keyword}&from={from_date}&sortBy=publishedAt&apiKey={api_key}&language={language}&page={page}'
            print(f"Fetching data for keyword '{keyword}' from page {page}...")

            response = requests.get(url)

            if response.status_code == 200:
                data = response.json()
                if data.get('status') == 'ok':
                    articles = data.get('articles', [])
                    descriptions = [preprocess_text(article.get('description')) for article in articles]  # Preprocess descriptions
                    all_descriptions.extend(descriptions)
                else:
                    print(f"Error: No articles found on page {page} for keyword '{keyword}'")
                    break
            else:
                print(f"Error: Request failed with status code {response.status_code} on page {page} for keyword '{keyword}'")
                break

            # Respect to API usage
            time.sleep(1)

    df = pd.DataFrame(all_descriptions, columns=['news'])
    df.to_pickle(file_path)

    print(f"Data saved to {file_path}")
    print(f"Number of rows created in the DataFrame: {df.shape[0]}")

    return df

api_key = ''  # Enter your NewsAPI key
file_path = '/content/drive/MyDrive/Projects/Deeplearning/all_news.pkl'
news_df = fetch_turkey_news(api_key, max_pages=5, file_path=file_path)

# Display the DataFrame
print(news_df.head())
last_row = news_df.iloc[-1]
print(last_row)

Fetching news for keyword: Turkey
Fetching data for keyword 'Turkey' from page 1...
Fetching data for keyword 'Turkey' from page 2...
Fetching data for keyword 'Turkey' from page 3...
Fetching data for keyword 'Turkey' from page 4...
Fetching data for keyword 'Turkey' from page 5...
Fetching news for keyword: Türkiye
Fetching data for keyword 'Türkiye' from page 1...
Fetching data for keyword 'Türkiye' from page 2...
Fetching data for keyword 'Türkiye' from page 3...
Fetching data for keyword 'Türkiye' from page 4...
Fetching data for keyword 'Türkiye' from page 5...
Fetching news for keyword: Ankara
Fetching data for keyword 'Ankara' from page 1...
Fetching data for keyword 'Ankara' from page 2...
Fetching data for keyword 'Ankara' from page 3...
Fetching data for keyword 'Ankara' from page 4...
Fetching data for keyword 'Ankara' from page 5...
Fetching news for keyword: Istanbul
Fetching data for keyword 'Istanbul' from page 1...
Fetching data for keyword 'Istanbul' from page 2...
Fe

##Step 1.b: Fetch Data with using RSS


In [ ]:
rss_feeds = [
"http://feeds.bbci.co.uk/news/world/rss.xml", #BBC News
"http://rss.cnn.com/rss/edition_world.rss", #CNN
"http://feeds.reuters.com/reuters/worldnews", #Reuters
"https://www.aljazeera.com/xml/rss/all.xml", #Al Jazeera English
"https://rss.nytimes.com/services/xml/rss/nyt/World.xml", #The New York Times
"https://www.theguardian.com/world/rss", #The Guardian
"http://feeds.washingtonpost.com/rss/world", #Washington Post
"https://www.npr.org/rss/rss.php?id=1004", #NPR
"https://www.bloomberg.com/feed/podcast/bloomberg", #Bloomberg
"http://www.ft.com/rss/home/us", #Financial Times
"https://abcnews.go.com/abcnews/internationalheadlines", #ABC News
"https://www.cbsnews.com/latest/rss/world", #CBS News
"http://feeds.nbcnews.com/nbcnews/public/world", #NBC News
"https://www.yahoo.com/news/rss/world", #Yahoo News
"https://apnews.com/apf-intlnews", #Associated Press
"http://feeds.feedburner.com/euronews/en/home/", #Euronews
"https://rss.dw.com/rdf/rss-en-all", #Deutsche Welle
"https://www.france24.com/en/rss", #France 24
"https://www.independent.co.uk/news/world/rss", #The Independent
]


turkey_keywords = [
    "Turkey", "Türkiye", "Ankara", "Istanbul", "Erdogan", "TUR", "Ottoman", "Cappadocia", "Bosphorus", "Galata",
    "Turkish", "Istanbul", "TURK", "Ataturk", "Marmara","Turkish lira","Galatasaray", "Turkish doner", "Turkey National Team"
    "Erdogan administration", "Turkish politics", "Turkish economy", "Turkish military", "Baykar"
    ]

def preprocess_text(text):
    # Use regex to filter out non-alphanumeric characters
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

def fetch_rss_news(keywords, rss_feeds):
    data = []

    for feed_url in rss_feeds:
        try:
            feed = feedparser.parse(feed_url)
            for entry in feed.entries:
                news_text = entry.get('description', entry.get('summary', ''))
                clean_text = preprocess_text(news_text)

                # Check if any keyword matches in the clean news text
                if any(keyword.lower() in clean_text.lower() for keyword in keywords):
                    data.append({'news': clean_text})
        except Exception as e:
            print(f"Error fetching RSS feed {feed_url}: {e}")
            continue

    return pd.DataFrame(data, columns=['news'])

# Load the existing dataset if necessary
#existing_file_path = "/content/drive/MyDrive/Projects/Deeplearning/all_news.pkl"
#all_news_data = pd.read_pickle(existing_file_path)

rss_news_df = fetch_rss_news(turkey_keywords, rss_feeds)

# Merge the new RSS news data
if not rss_news_df.empty:
    updated_news_data = pd.concat([all_news_data, rss_news_df], ignore_index=True)
else:
    updated_news_data = all_news_data
    print("No RSS news found that matches the keywords.")

new_file_path = "/content/drive/MyDrive/Projects/Deeplearning/all_news_updated.pkl"
updated_news_data.to_pickle(new_file_path)

print(f"Total news articles in the updated dataset: {len(updated_news_data)}")
print(f"Updated dataset saved to: {new_file_path}")

Total news articles in the updated dataset: 2306
Updated dataset saved to: /content/drive/MyDrive/Projects/Deeplearning/all_news_updated.pkl


#Step 2: Neural Machine Translation


In [ ]:
all_news_data = pd.read_pickle("/content/drive/MyDrive/Projects/Deeplearning/all_news_updated.pkl")

In [ ]:
# Function to clean HTML tags from text
def clean_html(text):
    if text is None:
        return ""
    return BeautifulSoup(text, "html.parser").get_text()

# Function to remove special characters using regex
def remove_special_characters(text):
    if text is None:
        return ""
    return re.sub(r'\W+', ' ', text)

# Combined cleaning function
def clean_text(text):
    text = clean_html(text)
    text = remove_special_characters(text)
    return text

In [ ]:
all_news_data['news'] = all_news_data['news'].apply(clean_text)

##Translation with using model

In [ ]:
model_name = 'Helsinki-NLP/opus-mt-tc-big-en-tr'
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)

def translate_to_turkish(text):
    translated = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translation = model.generate(**translated)
    translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)

    return translated_text

all_news_data['news_tr'] = all_news_data['news'].apply(translate_to_turkish)

print("\nTranslated Data Preview:")
print(all_news_data[['news', 'news_tr']].head())

translated_file_path = '/content/drive/MyDrive/Projects/Deeplearning/all_news_translated.pkl'
all_news_data.to_pickle(translated_file_path)

KeyboardInterrupt: 

## Translation Using Deepl

In [ ]:
API_KEY = 'YOUR_API_KEY'

translator = deepl.Translator(API_KEY)

In [ ]:
def translate_text_with_delay(text, target_language="TR", delay=0.1):
    try:
        result = translator.translate_text(text, target_lang=target_language)
        time.sleep(delay)
        return result.text
    except Exception as e:
        print(f"Error: {e}")
        return text

In [ ]:
all_news_data['news_tr']  = all_news_data['news'].apply(translate_text_with_delay, target_language='TR', delay=0.1)

Hata: Quota for this billing period has been exceeded, message: Quota Exceeded
Hata: Quota for this billing period has been exceeded, message: Quota Exceeded
Hata: Quota for this billing period has been exceeded, message: Quota Exceeded
Hata: Quota for this billing period has been exceeded, message: Quota Exceeded
Hata: Quota for this billing period has been exceeded, message: Quota Exceeded
Hata: Quota for this billing period has been exceeded, message: Quota Exceeded
Hata: Quota for this billing period has been exceeded, message: Quota Exceeded
Hata: Quota for this billing period has been exceeded, message: Quota Exceeded
Hata: Quota for this billing period has been exceeded, message: Quota Exceeded
Hata: Quota for this billing period has been exceeded, message: Quota Exceeded
Hata: Quota for this billing period has been exceeded, message: Quota Exceeded
Hata: Quota for this billing period has been exceeded, message: Quota Exceeded
Hata: Quota for this billing period has been exceede

KeyboardInterrupt: 

In [ ]:
translated_file_path = '/content/drive/MyDrive/Projects/Deeplearning/all_news_translated.pkl'
all_news_data.to_pickle(translated_file_path)

In [ ]:
all_news_translated = pd.read_pickle("/content/drive/MyDrive/Projects/Deeplearning/all_news_translated.pkl")

In [ ]:
all_news_translated.drop_duplicates(inplace=True)

In [ ]:
translated_file_path = '/content/drive/MyDrive/Projects/Deeplearning/all_news_translated.pkl'
all_news_translated.to_pickle(translated_file_path)

#Step 3: Sentiment analysis:

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import torch
import pandas as pd

all_news_data = pd.read_pickle("/content/drive/MyDrive/Projects/Deeplearning/all_news_translated.pkl")
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'

try:
    sentiment_model = AutoModelForSequenceClassification.from_pretrained(model_name)
    sentiment_tokenizer = AutoTokenizer.from_pretrained(model_name)
    print(f"Sentiment analysis model '{model_name}' loaded successfully.")
except Exception as e:
    print(f"Error loading sentiment analysis model: {e}")

# Function to predict sentiment (positive or negative)
def predict_sentiment(text):
    try:
        # Tokenize the text
        inputs = sentiment_tokenizer(
            text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )

        with torch.no_grad():
            outputs = sentiment_model(**inputs)
        logits = outputs.logits
        predicted_class_id = torch.argmax(logits, dim=-1).item()

        if predicted_class_id == 0:
            return 'negative'
        else:
            return 'positive'

    except Exception as e:
        return f"Error during sentiment prediction: {e}"

# Apply sentiment analysis to the 'news_tr' column
try:
    all_news_data['sentiment'] = all_news_data['news_tr'].apply(predict_sentiment)
    sentiment_file_path = '/content/drive/MyDrive/Projects/Deeplearning/news_sentiment.pkl'
    all_news_data.to_pickle(sentiment_file_path)

    print(all_news_data[['news', 'news_tr', 'sentiment']].head())
except Exception as e:
    print(f"Error during sentiment analysis application: {e}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Sentiment analysis model 'nlptown/bert-base-multilingual-uncased-sentiment' loaded successfully.
                                                news  \
0       Who fancies a festive game of Labour or Liz    
1  The Eurofighter program hit a new milestone on...   
2   Turkiye Garanti gets nod for green debt issuance   
3  A delegation from Turkey s main pro Kurdish DE...   
4  Dua Lipa wows fans with a glimpse of her engag...   

                                             news_tr sentiment  
0  Kim İşçi Partisi ya da Liz'in şenlikli bir oyu...  negative  
1  Eurofighter programı, Typhoon için bir orta öm...  positive  
2   Türkiye Garanti yeşil borç ihracı için onay aldı  positive  
3  Türkiye'nin Kürt yanlısı ana DEM partisinden b...  negative  
4  Dua Lipa, Callum Turner ile Noel'i şık bir şek...  positive  


### Analyze sentiment predictions


In [ ]:
sentiment_file = pd.read_pickle("/content/drive/MyDrive/Projects/Deeplearning/news_sentiment.pkl")

def analyze_sentiment_predictions(data):
    sentiment_counts = {'positive': 0, 'negative': 0}

    for _, row in data.iterrows():
        sentiment = row['sentiment']

        if sentiment == 'positive':
            sentiment_counts['positive'] += 1
        else:
            sentiment_counts['negative'] += 1


    print("Sentiment Counts:")
    for sentiment, count in sentiment_counts.items():
        print(f"  {sentiment}: {count}")

analyze_sentiment_predictions(sentiment_file)

Sentiment Counts:
  positive: 780
  negative: 1009


#Step 4: Methodology for sentiment analysis: Zero-Shot

In [ ]:
from transformers import pipeline

In [ ]:
all_news_data = pd.read_pickle("/content/drive/MyDrive/Projects/Deeplearning/all_news_translated.pkl")

In [ ]:
def get_sentiment(classifier, news):
    try:
        result = classifier(news, candidate_labels=['negative', 'positive'])
        return result['labels'][0]  # Return the label with the highest score
    except Exception as e:
        return "unknown"

In [ ]:
# Initialize classifiers for three different models
model_names = [
    "facebook/bart-large-mnli",  # Model 1
    "google-bert/bert-base-multilingual-cased",   # Model 2
    "MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli"          # Model 3
]


In [ ]:
classifiers = [pipeline("zero-shot-classification", model=model) for model in model_names]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Apply sentiment analysis for each model and add to separate columns
for i, classifier in enumerate(classifiers):
    model_name = model_names[i]
    all_news_data[f'{model_name}_sentiment'] = all_news_data['news_tr'].apply(lambda x: get_sentiment(classifier, x))


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
all_news_data.to_pickle("/content/drive/MyDrive/Projects/Deeplearning/all_news_zeroshot.pkl")

In [ ]:
all_news_data.head(5)

,news,news_tr,facebook/bart-large-mnli_sentiment,google-bert/bert-base-multilingual-cased_sentiment,MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli_sentiment
0,Who fancies a festive game of Labour or Liz,Kim İşçi Partisi ya da Liz'in şenlikli bir oyu...,negative,negative,positive
1,The Eurofighter program hit a new milestone on...,"Eurofighter programı, Typhoon için bir orta öm...",negative,positive,positive
2,Turkiye Garanti gets nod for green debt issuance,Türkiye Garanti yeşil borç ihracı için onay aldı,positive,negative,positive
3,A delegation from Turkey s main pro Kurdish DE...,Türkiye'nin Kürt yanlısı ana DEM partisinden b...,negative,negative,positive
4,Dua Lipa wows fans with a glimpse of her engag...,"Dua Lipa, Callum Turner ile Noel'i şık bir şek...",positive,negative,positive


### Analyze sentiment predictions

In [ ]:
all_news_zeroshot = pd.read_pickle("/content/drive/MyDrive/Projects/Deeplearning/all_news_zeroshot.pkl")

model_sentiment_columns = [
    'facebook/bart-large-mnli_sentiment',
    'google-bert/bert-base-multilingual-cased_sentiment',
    'MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli_sentiment',
]

def analyze_sentiment_predictions(data, model_columns):
    model_specific_counts = {}
    all_positive = 0
    all_negative = 0
    conflicting = 0

    for _, row in data.iterrows():
        sentiments = [row[col] for col in model_columns]

        # Count model-specific sentiment predictions
        for i, sentiment in enumerate(sentiments):
            model_name = model_columns[i]
            model_specific_counts.setdefault(model_name, {}).setdefault(sentiment, 0)
            model_specific_counts[model_name][sentiment] += 1

        # Check for consistency across models
        if all(s == 'positive' for s in sentiments):
            all_positive += 1
        elif all(s == 'negative' for s in sentiments):
            all_negative += 1
        else:
            conflicting += 1

    print("Model-Specific Sentiment Counts:")
    for model_name, sentiment_counts in model_specific_counts.items():
        print(f"\n{model_name}:")
        for sentiment, count in sentiment_counts.items():
            print(f"  {sentiment}: {count}")

    print("\nConsistency Across Models:")
    print(f"All models predicted positive sentiment: {all_positive}")
    print(f"All models predicted negative sentiment: {all_negative}")
    print(f"Conflicting predictions: {conflicting}")

analyze_sentiment_predictions(all_news_zeroshot, model_sentiment_columns)

Model-Specific Sentiment Counts:

facebook/bart-large-mnli_sentiment:
  negative: 1491
  positive: 297
  unknown: 1

google-bert/bert-base-multilingual-cased_sentiment:
  negative: 1373
  positive: 415
  unknown: 1

MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli_sentiment:
  positive: 1381
  negative: 407
  unknown: 1

Consistency Across Models:
All models predicted positive sentiment: 80
All models predicted negative sentiment: 293
Conflicting predictions: 1416


#Step 4.b: Methodology for sentiment analysis: Few Shot

In [ ]:
few_shot_examples = {
'positive': [
"Türkiye Eurofighter savaş uçağı alımına bir adım daha yaklaştı.",
"Türkiye Garanti yeşil borç ihracı için onay aldı",
"Çatışma Raporu Çad, Fransa ile askeri işbirliğine son verip Türkiye ile bağlarını güçlendiriyor ve Fransa'ya askerl
],

'negative': [
"Türkiye ekonomisi son yıllarda büyük bir krizle karşı karşıya, işsizlik oranları giderek artıyor",
"İstanbul trafiği her geçen gün daha kötü hale geliyor ve bu, insanların günlük yaşamını zorlaştırıyor",
"Türkiye eğitim sistemindeki eksiklikler nedeniyle birçok öğrenci yeterli eğitim alamıyor ve bu da gelecekteki iş gü
]
}

In [ ]:
candidate_labels = ['positive', 'negative']

In [ ]:
def get_sentiment_few_shot(classifier, news, positive_examples, negative_examples):
    # Define the prompt including the 3 positive and 3 negative examples
    prompt = f"Classify the following text as 'positive' or 'negative'.\n\n"

    # Add 3 positive examples to the prompt
    for example in positive_examples:
        prompt += f"Positive: {example}\n"

    # Add 3 negative examples to the prompt
    for example in negative_examples:
        prompt += f"Negative: {example}\n"

    prompt += f"Text: {news}\nClassification:"

    result = classifier(prompt, candidate_labels=candidate_labels)

    return result['labels'][0]

In [ ]:
# Initialize classifiers for three different models
model_names = [
    "facebook/bart-large-mnli",  # Model 1
    "google-bert/bert-base-multilingual-cased",   # Model 2
    "MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli"          # Model 3
]


In [ ]:
# Initialize the classifiers using the Hugging Face pipeline
classifiers = [pipeline("zero-shot-classification", model=model) for model in model_names]7

Device set to use cuda:0
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Device set to use cuda:0


In [ ]:
# apply sentiment analysis for each model
for i, classifier in enumerate(classifiers):
    model_name = model_names[i]
    all_news_data[f'{model_name}_sentiment_few_shot'] = all_news_data['news_tr'].apply(
        lambda x: get_sentiment_few_shot(classifier, x, few_shot_examples['positive'], few_shot_examples['negative'])
    )

In [ ]:
all_news_data.to_pickle("/content/drive/MyDrive/Projects/Deeplearning/all_news_zeroshot_fewshot.pkl")

In [ ]:
zeroshot_fewshot = pd.read_pickle("/content/drive/MyDrive/Projects/Deeplearning/all_news_zeroshot_fewshot.pkl")

zeroshot_fewshot.head()

,news,news_tr,facebook/bart-large-mnli_sentiment,google-bert/bert-base-multilingual-cased_sentiment,MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli_sentiment,facebook/bart-large-mnli_sentiment_few_shot,google-bert/bert-base-multilingual-cased_sentiment_few_shot,MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli_sentiment_few_shot
0,Who fancies a festive game of Labour or Liz,Kim İşçi Partisi ya da Liz'in şenlikli bir oyu...,negative,negative,positive,positive,positive,negative
1,The Eurofighter program hit a new milestone on...,"Eurofighter programı, Typhoon için bir orta öm...",negative,positive,positive,positive,positive,negative
2,Turkiye Garanti gets nod for green debt issuance,Türkiye Garanti yeşil borç ihracı için onay aldı,positive,negative,positive,positive,positive,negative
3,A delegation from Turkey s main pro Kurdish DE...,Türkiye'nin Kürt yanlısı ana DEM partisinden b...,negative,negative,positive,positive,positive,negative
4,Dua Lipa wows fans with a glimpse of her engag...,"Dua Lipa, Callum Turner ile Noel'i şık bir şek...",positive,negative,positive,positive,positive,negative


### Analyze sentiment predictions

In [ ]:
zeroshot_fewshot = pd.read_pickle("/content/drive/MyDrive/Projects/Deeplearning/all_news_zeroshot_fewshot.pkl")

model_sentiment_columns = [
    'facebook/bart-large-mnli_sentiment',
    'google-bert/bert-base-multilingual-cased_sentiment',
    'MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli_sentiment',
    'google-bert/bert-base-multilingual-cased_sentiment_few_shot',
    'facebook/bart-large-mnli_sentiment_few_shot',
    'MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli_sentiment_few_shot'
]

def analyze_sentiment_predictions(data, model_columns):
    model_specific_counts = {}
    all_positive = 0
    all_negative = 0
    conflicting = 0

    for _, row in data.iterrows():
        sentiments = [row[col] for col in model_columns]

        for i, sentiment in enumerate(sentiments):
            model_name = model_columns[i]
            model_specific_counts.setdefault(model_name, {}).setdefault(sentiment, 0)
            model_specific_counts[model_name][sentiment] += 1

        if all(s == 'negative' for s in sentiments):
            all_negative += 1
        else:
            all_positive += 1

    print("Model-Specific Sentiment Counts:")
    for model_name, sentiment_counts in model_specific_counts.items():
        print(f"\n{model_name}:")
        for sentiment, count in sentiment_counts.items():
            print(f"  {sentiment}: {count}")

    print("\nConsistency Across Models:")
    print(f"All models predicted positive sentiment: {all_positive}")
    print(f"All models predicted negative sentiment: {all_negative}")

analyze_sentiment_predictions(zeroshot_fewshot, model_sentiment_columns)

Model-Specific Sentiment Counts:

facebook/bart-large-mnli_sentiment:
  negative: 1491
  positive: 297
  unknown: 1

google-bert/bert-base-multilingual-cased_sentiment:
  negative: 1373
  positive: 415
  unknown: 1

MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli_sentiment:
  positive: 1381
  negative: 407
  unknown: 1

google-bert/bert-base-multilingual-cased_sentiment_few_shot:
  positive: 1778
  negative: 11

facebook/bart-large-mnli_sentiment_few_shot:
  positive: 1786
  negative: 3

MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli_sentiment_few_shot:
  negative: 1696
  positive: 93

Consistency Across Models:
All models predicted positive sentiment: 1789
All models predicted negative sentiment: 0


In [ ]:
def analyze_zero_shot_sentiment(data, zero_shot_columns):
    model_specific_counts = {}
    all_positive = 0
    all_negative = 0
    conflicting = 0

    for _, row in data.iterrows():
        sentiments = [row[col] for col in zero_shot_columns]

        for i, sentiment in enumerate(sentiments):
            model_name = zero_shot_columns[i]
            model_specific_counts.setdefault(model_name, {}).setdefault(sentiment, 0)
            model_specific_counts[model_name][sentiment] += 1

        if all(s == 'positive' for s in sentiments):
            all_positive += 1
        elif all(s == 'negative' for s in sentiments):
            all_negative += 1
        else:
            conflicting += 1

    print("Zero-Shot Sentiment Analysis:")
    print("Model-Specific Sentiment Counts:")
    for model_name, sentiment_counts in model_specific_counts.items():
        print(f"\n{model_name}:")
        for sentiment, count in sentiment_counts.items():
            print(f"  {sentiment}: {count}")

    print("\nConsistency Across Models:")
    print(f"All models predicted positive sentiment: {all_positive}")
    print(f"All models predicted negative sentiment: {all_negative}")
    print(f"Conflicting predictions: {conflicting}")


def analyze_few_shot_sentiment(data, few_shot_columns):
    model_specific_counts = {}
    all_positive = 0
    all_negative = 0
    conflicting = 0

    for _, row in data.iterrows():
        sentiments = [row[col] for col in few_shot_columns]

        for i, sentiment in enumerate(sentiments):
            model_name = few_shot_columns[i]
            model_specific_counts.setdefault(model_name, {}).setdefault(sentiment, 0)
            model_specific_counts[model_name][sentiment] += 1

        if all(s == 'positive' for s in sentiments):
            all_positive += 1
        elif all(s == 'negative' for s in sentiments):
            all_negative += 1
        else:
            conflicting += 1

    print("\nFew-Shot Sentiment Analysis:")
    print("Model-Specific Sentiment Counts:")
    for model_name, sentiment_counts in model_specific_counts.items():
        print(f"\n{model_name}:")
        for sentiment, count in sentiment_counts.items():
            print(f"  {sentiment}: {count}")

    print("\nConsistency Across Models:")
    print(f"All models predicted positive sentiment: {all_positive}")
    print(f"All models predicted negative sentiment: {all_negative}")
    print(f"Conflicting predictions: {conflicting}")

In [ ]:
zero_shot_columns = [
       'facebook/bart-large-mnli_sentiment',
       'google-bert/bert-base-multilingual-cased_sentiment',
       'MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli_sentiment'
   ]

In [ ]:
analyze_zero_shot_sentiment(zeroshot_fewshot, zero_shot_columns)


Zero-Shot Sentiment Analysis:
Model-Specific Sentiment Counts:

facebook/bart-large-mnli_sentiment:
  negative: 1491
  positive: 297
  unknown: 1

google-bert/bert-base-multilingual-cased_sentiment:
  negative: 1373
  positive: 415
  unknown: 1

MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli_sentiment:
  positive: 1381
  negative: 407
  unknown: 1

Consistency Across Models:
All models predicted positive sentiment: 80
All models predicted negative sentiment: 293
Conflicting predictions: 1416


In [ ]:
few_shot_columns = [
    'facebook/bart-large-mnli_sentiment_few_shot',
    'google-bert/bert-base-multilingual-cased_sentiment_few_shot',
    'MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli_sentiment_few_shot'
]

In [ ]:
analyze_few_shot_sentiment(zeroshot_fewshot, few_shot_columns)


Few-Shot Sentiment Analysis:
Model-Specific Sentiment Counts:

facebook/bart-large-mnli_sentiment_few_shot:
  positive: 1786
  negative: 3

google-bert/bert-base-multilingual-cased_sentiment_few_shot:
  positive: 1778
  negative: 11

MoritzLaurer/multilingual-MiniLMv2-L6-mnli-xnli_sentiment_few_shot:
  negative: 1696
  positive: 93

Consistency Across Models:
All models predicted positive sentiment: 90
All models predicted negative sentiment: 0
Conflicting predictions: 1699
